In [1]:
import pandas as pd
import numpy as np
import collections
import itertools
import string
from collections import Counter
from itertools import chain
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
    
#nltk.download()

In [6]:
import os
os.chdir('E:\Spring 2020\IDS 561')
data=open('Pride_and_Prejudice.txt',encoding='utf-8',mode='r')
#print(data.read())
content=data.readlines()


### Dividing data into two parts to pass it to two Mappers

In [18]:
Data=list(filter('\n'.__ne__,content))
Part1=''.join(Data[0:5000])
Part2=''.join(Data[5000:])

### Function for making a list of words and cleaning the textfile

In [23]:
def Tokenizefunction(rawdata):
    list1=[]
    list1=[i.lower() for i in word_tokenize(rawdata) if i.isalpha()]
    return(list1)


In [24]:
#Calling Tokenizefunction
Data1=Tokenizefunction(Part1)
Data2=Tokenizefunction(Part2)

In [39]:
#print(Data1)
#print(Data2)

### Mapper function

In [26]:
def Mapperfunction(rawdata):
    list=[]
    for i in rawdata:
        list.append([i,1])
    return(list)
    

### Multithreading Mapper function

In [27]:
import concurrent.futures
def MultithreadingMapper(rawdata1,rawdata2):
    with concurrent.futures.ThreadPoolExecutor() as multithreader:
            text1 = multithreader.submit(Mapperfunction, rawdata1)
            text2 = multithreader.submit(Mapperfunction, rawdata2)
            Return1 = text1.result()
            Return2 = text2.result()
            
    Mapperoutput=Return1 + Return2
    return(Mapperoutput)
    

In [28]:
#Calling MultithreadingMapper Function on tokenized data
Mapper_Output=MultithreadingMapper(Data1,Data2)

In [40]:
#print(Mapper_Output)

In [29]:
 #Sorting data.If first character of word is less than n then in sort1 else in sort2
def sorting(rawdata):
    rawdata=sorted(rawdata)
    sort1=[]
    sort2=[]

    for i in rawdata:
        if(i[0][0] <'n'):    
            sort1.append(i)
        else:
            sort2.append(i)
    return(sort1,sort2)

In [30]:
 #Calling Sorting function and Partitioning the Key,Value Pair
Sorted1,Sorted2=sorting(Mapper_Output)

### Reducer Function

In [31]:

def Reducer(rawdata):
    Frequency=collections.defaultdict(list)
    i=0
    while(i <len(rawdata)):
        count=0
        for j in range(i,len(rawdata)):
            if(rawdata[i][0] == rawdata[j][0]):
                count+=1
        Frequency[rawdata[i][0]]=count
        i=i+count
    return(Frequency)
    
    

# Multithreading Reducer Function

In [33]:
def Multithreading_Reducer(rawdata,rawdata1): 
        with concurrent.futures.ThreadPoolExecutor() as execute:
            thread3 = execute.submit(Reducer, rawdata)
            thread4 = execute.submit(Reducer, rawdata1)
            Return3 = thread3.result()
            Return4 = thread4.result()
            return(Return3,Return4) 

In [34]:
Reducer_Output1=collections.defaultdict(list)
Reducer_Oulput2=collections.defaultdict(list)

In [35]:
#Calling Multithreading Reducer Function
Reducer_Output1,Reducer_Output2=Multithreading_Reducer(Sorted1,Sorted2)

Reducer_combined_Output = {**Reducer_Output1, **Reducer_Output2}

### Reducer Output Function

In [41]:
#for i in Reducer_combined_Output.keys():
#    print(i,Reducer_combined_Output[i])
    

In [64]:
#Storing Results in CSV File
with open('C:\Users\Suhail\Big Data Analytics_Asg1.csv', 'w') as f:
    for i in Reducer_combined_Output.keys():
        f.write("%s,%s\n"%(i,Reducer_combined_Output[i]))